In [ ]:
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,classification_report,f1_score
from collections import Counter

In [ ]:
attr = pd.read_csv('attr.txt', sep='\t',
                           names=['link', 'length', 'direction', 'path_class', 'speed_class', 'LaneNum', 'speed_limit',
                                  'level', 'width'], header=None)

In [ ]:
train = pd.read_csv('is_train_data/is_train_2019070' + str(1) + '.txt')
test = pd.read_csv('is_test.csv')
train = train.merge(attr, on='link', how='left')
test = test.merge(attr, on='link', how='left')

In [ ]:
id_col = 'link'
use_train_feats = [i for i in train.columns if i not in ['link', 'label', 'current_slice_id', 'future_slice_id', 'label_pred']]

In [ ]:
train_user_id = train[id_col].unique()

In [ ]:
data_0 = train.loc[train['label'].isin(['0'])]
data_1 = train.loc[train['label'].isin(['1'])]
data_2 = train.loc[train['label'].isin(['2'])]

In [ ]:
data_01 = pd.concat([data_0,data_1])
data_12 = pd.concat([data_0,data_2])
data_02 = pd.concat([data_1,data_2])

In [ ]:
clf_01 = tree.DecisionTreeClassifier\
            (class_weight="balanced", criterion='entropy', max_depth=10, splitter="random",random_state=0,min_weight_fraction_leaf=0.000001)

In [ ]:
clf_02 = tree.DecisionTreeClassifier\
            (class_weight="balanced", criterion='entropy', max_depth=10, splitter="random",random_state=0,min_weight_fraction_leaf=0.000001)

In [ ]:
clf_12 = tree.DecisionTreeClassifier\
            (class_weight="balanced", criterion='entropy', max_depth=10, splitter="random",random_state=0,min_weight_fraction_leaf=0.000001)

In [ ]:
X_train_01= np.array(data_01.loc[:,use_train_feats])
X_train_02= np.array(data_12.loc[:,use_train_feats])
X_train_12= np.array(data_02.loc[:,use_train_feats])
y_train_01 = np.array(data_01['label']).reshape(-1,1)
y_train_02 = np.array(data_12['label']).reshape(-1,1)
y_train_12 = np.array(data_02['label']).reshape(-1,1)

In [ ]:
clf_01.fit(X_train_01,y_train_01)
clf_02.fit(X_train_02,y_train_02)
clf_12.fit(X_train_12,y_train_12)

In [ ]:
train = pd.read_csv('is_train_data/is_train_2019070' + str(2) + '.txt')
train = train.merge(attr, on='link', how='left')

In [ ]:
X_ver= np.array(train.loc[:,use_train_feats])
y_ver = np.array(train['label']).reshape(-1,1)

In [ ]:
res01 = clf_01.predict(X_ver)
res02 = clf_02.predict(X_ver)
res12 = clf_12.predict(X_ver)

In [ ]:
sum_res = np.vstack((np.vstack(((res01,res02))),res12))

In [ ]:
sum_res = sum_res.transpose()

In [ ]:
sum_res

In [ ]:
result = [Counter(sum_res[i,:]).most_common(1)[0][0] for i in range(len(sum_res))]

In [ ]:
result = np.array(result)

In [ ]:
print(classification_report(y_ver.reshape(-1),result))

In [ ]:
report = f1_score(y_ver, result, average=None)
print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)